Dans l'univers du développement logiciel, les tests sont omni-présents. Ils permettent de vérifier que le logiciel ou l'application développée adopte correctement le comportement attendu, ne produit pas de bugs ou s'intègre efficacement dans un environnement existant.

Mais comment transposer tous ces tests de développement logiciel au cas où l'on entraîne et fait intervenir des modèles de Machine Learning ?

<blockquote><p>🙋 <b>Ce que nous allons faire</b></p>
<ul>
    <li>Comprendre pourquoi il est important de tester son code et son modèle</li>
    <li>Appliquer les tests usuels de développement logiciel</li>
</ul>
</blockquote>

<img src="https://media.giphy.com/media/Rd6sn03ncIklmprvy6/giphy.gif" />

## Tester des applications

Avant de rentrer dans le détails des tests d'algorithmes de Machine Learning, décrivons tout d'abord les bonnes pratiques héritées du développement logiciel.

### Tests logiciels

Dans ce contexte, une suite de tests inclut habituellement trois composantes.

- Les **tests unitaires**, où l'on s'assurer qu'une portion atomique du code fonctionne correctement (par exemple, une fonction). En règle générale, ce sont des tests rapides et faciles à mettre en place.
- Les **tests de régression**, où l'on doit s'assurer que le développement d'une nouvelle fonctionnalité ne va pas faire survenir un bug déjà rencontré par le passé.
- Les **tests d'intégration**, où on cherche à voir si la fonctionnalité développée va être correctement intégré dans l'application sans générer des erreurs dues à son interaction avec d'autres composantes. Ces erreurs sont en pratique plus difficiles à prévenir, d'où la difficulté de construire des tests d'intégration efficaces.

Dans les faits, les bonnes pratiques nécessitent de suivre plusieurs conventions. En travail collaboratif, notamment avec `git`, les règles de base suivantes sont appliquées.

- Ne **jamais fusionner de branches** si les tests ne sont pas valides.
- **Toujours écrire des tests** pour de nouvelles fonctionnalités.
- Lorsque l'on corrige un bug, **toujours écrire le test** et l'appliquer sur la correction.

### Tests de modèles de Machine Learning

Essayons maintenant de transposer ce que nous venons de voir pour tester les modèles de Machine Learning. Une fois un modèle de Machine Learning calibré, nousz souhaiterions obtenir un rapport d'évaluation contenant les informations suivantes.

- Performances avec des métriques définies sur des sous-ensembles (`X_test` par exemple).
- Graphes de validation : courbe PR, courbe ROC, densité des classes, courbe de calibration.
- Audit du modèle avec des modèles d'interprétabilité (PDP, valeurs de Shapley).
- Sous-population où le modèle génère des faux-positifs ou faux-négatifs avec un fort degré de confiance.

Par ailleurs, on y retrouve également d'autres bonnes pratiques qui s'inscrivent toujours dans une logique de démarche de qualité.

- **Toujours sauvegarder** les hyper-paramètres, sous-échantillons utilisés et le modèle entraîné.
- Mettre à jour un environnement de production avec **un modèle aux meilleures performances** ou selon un seuil minimal.

Face à ces besoins de tester, nous pouvons voir que calculer des performances sur un sous-échantillon ou afficher des courbes n'est pas suffisant pour s'assurer que le modèle est « valide ». Pour les systèmes de Machine Learning, nous devrions effectuer deux méthodes en parallèle.

- **L'évaluation de modèle**, où l'on calcule ses performances, audite son fonctionnement et affiche des courbes.
- Le **test de modèle** où l'on développe des tests explicites pour vérifier que le comportement du modèle est bien celui attendu.

## Tests unitaires

Commençons par introduire les tests unitaires avec `pytest`. Il s'agit d'une librairie qui permet de **faciliter la mise en place et l'exécution** des tests de code sous Python. Bien que les tests unitaires puissent être réalisés *from scratch*, `pytest` améliore la productivité et apporte des fonctionnalités très utiles.

Testons la librairie sur le premier fichier suivant. Nous avons codé la fonction `argmax` qui cherche à obtenir la position du plus grand élément d'une liste. Nous codons également la fonction `test_argmax` qui va tester unitairement la fonction `argmax` sur plusieurs exemples : cela reflète du comportement attendu de la fonction.

In [5]:
%%writefile /tmp/pytest_1.py
def argmax(liste):
    if len(liste) == 0:
        return None
    
    idx_max = 0
    value_max = liste[0]
    for i, x in enumerate(liste):
        if x > value_max:
            value_max = x
            idx_max = i
    return idx_max

def test_argmax():
    assert argmax([5, 8, 2, 9, 6, 3]) == 3
    assert argmax([7]) == 0
    assert argmax([]) == None

Exécutons le code avec `pytest` en spécifiant le chemin d'accès au fichier.

In [6]:
!pytest /tmp/pytest_1.py

En exécutant cette commande, `pytest` effectue une **découverte automatique** des tests.

- Il va d'abord rechercher tous les fichiers dont le nom commence par `test*` si on lui fournit un dossier.
- Pour chaque classe/fonction du fichier, si l'objet commence par `test*`, alors ce dernier sera instancié (dans le cas d'une fonction) et les fonctions seront exécutées (pour les deux).

Cette découverte des tests permet de simplifier la mise en place des tests : plus besoin de spécifier tous les tests dans un fichier, qui lui-même effectue des importations. Nous pouvons imaginer que pour chaque *module*, il y ait un fichier `test.py` qui regroupe tous les tests unitaires liés à ce module. De manière générale, il est plus approprié de créer un fichier spécifique pour les tests unitaires plutôt que de les insérer dans le code qui fournit la logique à l'application.

C'est de cette manière que `pytest` exécute naturellement la fonction `test_argmax` sans avoir eu besoin de la spécifier comme argument. Dans certains cas, nous pouvons être amené à éviter volontairement l'exécution d'une fonction. Dans ce cas, il suffit d'ajouter le décorateur `pytest.mark.skip`.

In [8]:
%%writefile /tmp/pytest_1.py
import pytest

def argmax(liste):
    if len(liste) == 0:
        return None
    
    idx_max = 0
    value_max = liste[0]
    for i, x in enumerate(liste):
        if x > value_max:
            value_max = x
            idx_max = i
    return idx_max

@pytest.mark.skip
def test_argmax():
    assert argmax([5, 8, 2, 9, 6, 3]) == 3
    assert argmax([7]) == 0
    assert argmax([]) == None

In [9]:
!pytest /tmp/pytest_1.py

Comme nous pouvons le voir, 100% des tests ont réussi car le seul test présent a été ignoré (*skipped*). Voyons maintenant un autre fichier Python dont le test unitaire va volontairement générer une erreur.

In [10]:
%%writefile /tmp/pytest_2.py
def argmin(liste):
    if len(liste) == 0:
        return None
    
    idx_min = 0
    value_min = liste[0]
    for i, x in enumerate(liste):
        if x < value_min:
            value_min = x
            idx_min = i + 1
    return idx_min

def test_argmin():
    assert argmin([5, 8, 2, 9, 6, 3]) == 2
    assert argmin([7]) == 0
    assert argmin([]) == None

In [11]:
!pytest /tmp/pytest_2.py

D'après la sortie générée par `pytest`, les tests du fichier `/tmp/pytest_2.py` ont échoués. Si l'on regarde en détaille l'exécution de `test_argmin`, nous avons un `assert 3 == 2`, ce qui signifie que notre test unitaire a échoué. Corrigeons la fonction `argmin` et ajoutons la fonction `argmax` avec son test unitaire associé.

In [12]:
%%writefile /tmp/pytest_2.py
def argmin(liste):
    if len(liste) == 0:
        return None
    
    idx_min = 0
    value_min = liste[0]
    for i, x in enumerate(liste):
        if x < value_min:
            value_min = x
            idx_min = i
    return idx_min

def argmax(liste):
    if len(liste) == 0:
        return None
    
    idx_max = 0
    value_max = liste[0]
    for i, x in enumerate(liste):
        if x > value_max:
            value_max = x
            idx_max = i
    return idx_max

def test_argmin():
    assert argmin([5, 8, 2, 9, 6, 3]) == 2
    assert argmin([7]) == 0
    assert argmin([]) == None
    
def test_argmax():
    assert argmax([5, 8, 2, 9, 6, 3]) == 3
    assert argmax([7]) == 0
    assert argmax([]) == None

In [13]:
!pytest /tmp/pytest_2.py -v

Le paramètre `-v` permet d'afficher plus de détails concernant les tests. Puisque deux fonctions sont nommées `test*`, il y a deux tests effectués par `pytest`. Cette option permet d'obtenir un détail pour chaque test codé, simplifiant ensuite le déboggage de l'application. 

En pratique, les tests unitaires doivent être exécutés une fois les données envoyés vers le dépôt Git. En revanche, il est déconseillé de les exécuter lors du pre-commit, car ce dernier doit être rapide. Les tests unitaires, notamment ceux incluant des tests pour les modèles, peuvent prendre du temps ce qui n'est pas conseillé pour les pre-commits.

### Les fixtures

Imaginons que l'on souhaite utiliser des données/paramètres uniquement pour les tests unitaires. Si l'on regarde bien, les deux fonctions `test_argmin` et `test_argmax` utilisent les mêmes listes pour tester les deux fonctions. Nous pourrions tout à fait définir des catalogues de référence pour les tests unitaires qui seront utilisés à chaque fois. C'est à cela que servent **les fixtures**.

Regardons le code suivant qui n'utilise pas de fixture. Nous allons simplement créer une liste `test_data` qui sera utilisée par les deux fonctions de test.

In [14]:
%%writefile /tmp/pytest_2.py

# Pas bien !
test_data = [5, 8, 2, 9, 6, 3]

def argmin(liste):
    if len(liste) == 0:
        return None
    
    idx_min = 0
    value_min = liste[0]
    for i, x in enumerate(liste):
        if x < value_min:
            value_min = x
            idx_min = i
    return idx_min

def argmax(liste):
    if len(liste) == 0:
        return None
    
    idx_max = 0
    value_max = liste[0]
    for i, x in enumerate(liste):
        if x > value_max:
            value_max = x
            idx_max = i
    return idx_max

def test_argmin():
    assert argmin(test_data) == 2
    
def test_argmax():
    assert argmax(test_data) == 3

In [15]:
!pytest /tmp/pytest_2.py -v

Bien que le test ait fonctionné, cela n'est pas une bonne pratique, car nous allons obligatoirement définir cette variable globale en mémoire à chaque exécution du code, alors qu'elle n'est utilisée que pour les tests unitaires. Dans ce cas de figure, il est préférable de créer des fixtures.

Les fixtures définissent un environnement dans lequel nous allons pouvoir tester notre code. Dans beaucoup de situations, il nous faut initialiser certaines variables avant de lancer les tests unitaires. Les fixtures sous `pytest` sont des fonctions qui sont utilisés comme **paramètres** des fonctions de tests unitaires.

Regardons le code suivant.

In [17]:
%%writefile /tmp/pytest_2.py
import pytest

@pytest.fixture
def test_data():
    return [5, 8, 2, 9, 6, 3]

def argmin(liste):
    if len(liste) == 0:
        return None
    
    idx_min = 0
    value_min = liste[0]
    for i, x in enumerate(liste):
        if x < value_min:
            value_min = x
            idx_min = i
    return idx_min

def argmax(liste):
    if len(liste) == 0:
        return None
    
    idx_max = 0
    value_max = liste[0]
    for i, x in enumerate(liste):
        if x > value_max:
            value_max = x
            idx_max = i
    return idx_max

def test_argmin(test_data):
    assert argmin(test_data) == 2
    
def test_argmax(test_data):
    assert argmax(test_data) == 3 

Tout d'abord, nous définissons la fonction `test_data` comme fixture à l'aide du décorateur de fonctions `@pytest.fixture`. Cette fonction va renvoyer une liste qui correspond à la liste de référence pour tester les deux fonctions. Ensuite, dans les fonctions de tests unitaires, nous allons récupérer comme paramètre cette même fonction `test_data`. Mais attention : lorsque l'on exécutera `pytest`, ce dernier va automatiquement remplacer le paramètre `test_data` (qui est supposé être une fonction car fixture) par le résultat de cette fonction.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/tests4.png" />

Ainsi, à chaque exécution de `pytest`, ce sera en réalité `test_data()` qui sera passé comme paramètre pour les fonctions `test_argmin` et `test_argmax` (et non la fonction `test_data` elle-même). Cette méthode permet d'instancier plus efficacement les initialisations pour les tests, sans compromettre le reste du code qui lui n'aura pas besoin des tests dans un environnement de production.

Exécutons maintenant `pytest`.

In [18]:
!pytest /tmp/pytest_2.py -v

Tout a correctement fonctionné. L'intérêt de ce système est de pouvoir ensuite centraliser l'initialisation des variables et des données pour les tests, évitant ainsi les duplicata de codes que l'on connaît déjà bien hors des tests.

Maintenant que nous avons vu les points essentiels de `pytest`, nous pouvons dorénavant intégrer les tests unitaires dans notre projet Kedro. Et un avantage non négligeable est que Kedro supporte nativement `pytest` pour les tests unitaires : il dispose même de la commande `kedro test`. 🙂

## Intégration des tests unitaires dans Kedro

Intégrons les tests unitaires et du modèle dans notre projet Kedro. En regardant la structure du projet, nous pouvons observer le dossier `src/tests` qui contient le fichier `test_run.py`.

Pour exécuter proprement les tests avec Kedro, il faut que la structure des fichiers des tests soit identique à celle utilisée dans `src/purchase_predict`. Nous devons donc créer deux dossiers `loading`, `training` et `processing` dans `src/tests/pipelines` pour répliquer l'architecture à l'identique.

Commençons par le dossier `loading` qui charge les fichiers CSV depuis Cloud Storage. Au préalable, nous allons installer les dépendances de Kedro pour effectuer les tests unitaires (qui contient `pytest` notamment).

### Tests sur les nodes

Avant de développer nos tests unitaires, créons sur le bucket Cloud Storage des **données de tests**. Dans le dossier `primary/` du bucket, nous allons créer un dossier `data-test.csv/`.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/tests1.png" />

Ensuite, pour alimenter ce dossier, nous allons copier deux fichiers CSV déjà présents dans `data.csv/` vers `data-test.csv/`.

<img src="https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/training/ml_engineer_facebook/img/tests2.png" />

Habituellement, avec Kedro, nous pouvons effectuer deux séries de tests.

- Les tests sur les nodes et les fonctions qu'utilisent les nodes. Par exemple, pour l'entraînement, le fichier `nodes.py` contient des fonctions qui ne sont pas directement utilisés par les nodes mais qui sont appelés par les fonctions des nodes.
- Les tests sur les pipelines, permettant de les tester en fonction de plusieurs formats d'entrée ou sous forme d'exécution partielles.

Créons tout d'abord le fichier `test_nodes.py`. Dans le pipeline `loading`, seule la fonction `load_csv_from_bucket` est présente : nous allons uniquement tester cette dernière.

Nous définissons la fonction `test_load_csv_from_bucket` avec les mêmes paramètres que la fonction `load_csv_from_bucket`.

> ❓ Mais nous n'avons pas défini les fixtures ici ?

En effet, il faudrait que les paramètres `project_id` et `primary_folder` soient des fixtures avec des fonctions de même nom. Or, ici, nous n'en avons pas créée. Il y a une raison à cela : plus tard, nous allons également créer un fichier de test pour le pipeline. Pour éviter des redondances de définitions de fixtures, nous allons définir les fixtures dans un fichier spécifique, qui derrière sera automatiquement exécuté par `pytest`.

D'après <a href="https://docs.pytest.org/en/stable/fixture.html#conftest-py-sharing-fixture-functions" target="_blank">la documentation</a> de `pytest` sur les fixtures, nous pouvons les centraliser dans un fichier nommé `conftest.py` qui sera automatiquement exécuté avant les tests unitaires. Nous en créons un dans le dossier `loading`.

L'argument `scope="module"` permet de spécifier que les fixtures seront accessibles à l'intérieur de `purchase_predict`. Il ne reste plus qu'à lancer les test avec Kedro.

Au tout début, `pytest` exécute le test sur `test_run.py`, qui montre un exemple de test unitaire avec Kedro. Ensuite, il exécute le seul autre fichier de test présent `test_nodes.py`. Puisqu'il n'y a aucun problème, cela signifie que le code n'a pas généré d'erreurs et que, en théorie, nous avons correctement réussi à implémenter la fonction de test avec Kedro. C'est alors que nous pouvons rajouter des tests et des conditions dans la fonction.

### Tests sur les pipelines

En plus de tests unitaires sur les nodes, il est également possible d'effectuer des tests unitaires sur les pipelines. Cela permet, par exemple, de s'assurer du bon déroulement du pipeline en fonction de plusieurs situations (données incomplètes ou manquantes, mauvaise configuration de paramètres). En respectant le même principe que pour les nodes, nous allons créer le fichier `test_pipeline.py`.

Nous récupérons la fonction `create_pipeline` permettant de créer le pipeline que nous souhaitons tester. Dans le test unitaire, nous instancions un `SequentialRunner`, qui exécutera le pipeline de manière séquentielle. Ensuite, nous créons une instance du pipeline et enfin nous exécuter ce dernier. Remarquons la variable `catalog_test` : il s'agit d'un catalogue de données spécifiquement crée pour le test. Plutôt que d'utiliser celui par défaut dans le fichier `catalog.yml`, nous allons pouvoir spécifier des données propres aux tests qui ne va pas perturber le catalogue déjà présent.

Le catalogue de données représente lui aussi une fixture que nous rajoutons dans `conftest.py`.

Cette fonction retourne un `DataCatalog` qui sera envoyé en entrée au pipeline.

<div class="alert alert-block alert-warning">
    Il faut respecter les noms des variables spécifiés dans le pipeline. 
</div>

Pour rappel, le pipeline `loading` était défini de la manière suivante.

Là-aussi, `pytest` remplacera `catalog_test` par la fixture associée et permettra d'initialiser correctement l'environnement de test.

Le pipeline a été exécuté sans problème. Nous pouvons là-aussi rédiger des tests pour le pipeline, qui en soit seront quasi-identiques à ceux du node car ce pipeline ne contient qu'un seul node et ce dernier n'appelle pas d'autres fonctions.

## ✔️ Conclusion

Peut-être il s'agit de ton premier test unitaire avec Python : dans tous les cas, tu sais maintenant en écrire, et c'est une très bonne pratique !

- Nous avons vu pourquoi les tests logiciels étaient indispensables.
- Nous avons rédigé plusieurs tests unitaires pour le pipeline de collecte des données.

> ➡️ Il nous reste maintenant à définir et rédiger les <b>tests sur le modèle de Machine Learning</b>.